In [2]:
import pandas as pd
import numpy as np
import uuid

#extracts data from excel file into a pandas dataframe 
data = pd.read_excel('sample ClassSched-CS-S25.xlsx', header=1)

#lool at the data
print(data)

   College Acad Org Subject  Catalog  Section                           Title  \
0     ENGR    R1260      CS      135     1001              Computer Science I   
1     ENGR    R1260      CS      135     1101              Computer Science I   
2     ENGR    R1260      CS      135     1102              Computer Science I   
3     ENGR    R1260      CS      135     1103              Computer Science I   
4     ENGR    R1260      CS      135     1104              Computer Science I   
5     ENGR    R1260      CS      135     1105              Computer Science I   
6     ENGR    R1260      CS      135     1106              Computer Science I   
7     ENGR    R1260      CS      202     1001             Computer Science II   
8     ENGR    R1260      CS      202     1101             Computer Science II   
9     ENGR    R1260      CS      202     1102             Computer Science II   
10    ENGR    R1260      CS      202     1103             Computer Science II   
11    ENGR    R1260      CS 

In [3]:
#populates dataframe for college_df, creating unique id for each row
college_df = data[['College']].drop_duplicates().rename(columns={'College': 'CollegeName'})
college_df['CollegeID'] = range(1, len(college_df) + 1)
print(college_df)

  CollegeName  CollegeID
0        ENGR          1


In [4]:

#populates dataframe for instructor_df, creating unique id for each row
instructor_df = data[['Instructor First Name', 'Instructor Last Name']].drop_duplicates().rename(columns={
                                                                                                'Instructor First Name': 'InstructorFirstName', 
                                                                                                'Instructor Last Name': 'InstructorLastName'})
instructor_df['InstructorID'] = range(1, len(instructor_df) + 1)

print(instructor_df)

   InstructorFirstName InstructorLastName  InstructorID
0                 Erin              Keith             1
7              Bashira        Akter Anima             2
14               Nancy        Latourrette             3
16              Mircea          Nicolescu             4
19               Diana               Moss             5
20              Joshua               Dahl             6
21                Sara              Davis             7
22               David        Feil-Seifer             8
23                 Lei               Yang             9
24              Shamik           Sengupta            10
25            Christos       Papachristos            11
26             Zachary           Estreito            12
27             William            Doherty            13
28                Hung                 La            14
30              Jordan           Hastings            15
31              Monica          Nicolescu            16
32              George              Bebis       

In [5]:
#populates dataframe for room_df, creating unique id for each row
room_df = data[['Room', 'Room Capacity']].drop_duplicates().rename(columns={'Room': 'RoomNo', 'Room Capacity': 'RoomCapacity'})

room_df['RoomID'] = range(1, len(room_df) + 1)

print(room_df)

      RoomNo  RoomCapacity  RoomID
0    SEM 101           254       1
1   WPEB 100            58       2
7   WPEB 130           210       3
13   SEM 234            70       4
14   LME 321            32       5
15  DMSC 104            83       6
16  DMSC 103            83       7
17   SEM 261            49       8
23   CFA 153            68       9
24  WPEB 200            36      10
26  SEM 231C            41      11
27    MS 321            48      12
28   SEM 347            40      13
29    AB 102            93      14
31  DMSC 110           464      15
33  DMSC 102            69      16
34  DMSC 105            83      17


In [6]:
status_df = data[['Class Stat']].drop_duplicates().rename(columns={'Class Stat': 'StatusCode'})

status_df['StatusID'] = range(1, len(status_df) + 1)

print(status_df)

  StatusCode  StatusID
0          A         1


In [7]:
#populates dataframe for class
class_df = data[['Title', 'Catalog', 'Subject', 'Section', 'Enrollment Capacity', 'College']].drop_duplicates().rename(columns={'Enrollment Capacity': 'Enrollment_Capacity'})
class_df['College_ID'] = college_df['CollegeID']

#adds CollegeID to class_df from college_df
class_df = class_df.merge(college_df[['CollegeName', 'CollegeID']], 
                          
                          #Join conditions
                          left_on='College', 
                          right_on='CollegeName', 
                          
                          #join type
                          how='left')

class_df = class_df.drop('CollegeName', axis=1)

print(class_df)

                             Title  Catalog Subject  Section  \
0               Computer Science I      135      CS     1001   
1               Computer Science I      135      CS     1101   
2               Computer Science I      135      CS     1102   
3               Computer Science I      135      CS     1103   
4               Computer Science I      135      CS     1104   
5               Computer Science I      135      CS     1105   
6               Computer Science I      135      CS     1106   
7              Computer Science II      202      CS     1001   
8              Computer Science II      202      CS     1101   
9              Computer Science II      202      CS     1102   
10             Computer Science II      202      CS     1103   
11             Computer Science II      202      CS     1104   
12             Computer Science II      202      CS     1105   
13           Computer Organization      219      CS     1001   
14  Digital Forensics Fundamentals      

In [ ]:
section_df = data[[
    'Class Nbr', 'Component', 'Room Capacity', 'Combined?', 'Waitlist Capacity',
    'Waitlist Total', 'Prgrss Unt', 'Class Stat', 'Session', 'Instruction Mode'
]].drop_duplicates().copy()

section_df.rename(columns={
    'Class Nbr': 'SectionClassID',
    'Component': 'Component',
    'Room Capacity': 'Room',
    'Combined?': 'isCombined',
    'Waitlist Capacity': 'WaitlistCapacity',
    'Waitlist Total': 'WaitlistTotal',
    'Prgrss Unt': 'ProgressUnit',
    'Class Stat': 'StatusID',
    'Session': 'Session',
    'Instruction Mode': 'InstructionMode'
}, inplace=True)

section_df['isCombined'] = section_df['isCombined'].map({'Yes': True, 'No': False})

print(section_df)


    SectionClassID Component  Room  isCombined  WaitlistCapacity  \
0            24638       LEC   254       False               999   
1            24639       LAB    58       False                 0   
2            24640       LAB    58       False               999   
3            24646       LAB    58       False               999   
4            24654       LAB    58       False               999   
5            24751       LAB    58       False               999   
6            24752       LAB    58       False               999   
7            24641       LEC   210       False               999   
8            24642       LAB    58       False               999   
9            24643       LAB    58       False               999   
10           24644       LAB    58       False               999   
11           24645       LAB    58       False               999   
12           24714       LAB    58       False               999   
13           24682       LEC    70       False  

In [ ]:
data['InstructorFullName'] = data['Instructor First Name'].str.strip() + ' ' + data['Instructor Last Name'].str.strip()
instructors = data[['InstructorFullName']].drop_duplicates().reset_index(drop=True)
instructors['InstructorID'] = instructors.index + 1 

data = data.merge(instructors, on='InstructorFullName', how='left')

data['SectionClassID'] = data['Class Nbr']

section_instructor = data[['SectionClassID', 'InstructorID']].drop_duplicates()

section_instructor = section_instructor.sort_values(by=['SectionClassID', 'InstructorID']).reset_index(drop=True)

print(section_instructor)


    SectionClassID  InstructorID
0            24635             1
1            24636             5
2            24637            11
3            24638             1
4            24639             1
5            24640             1
6            24641             2
7            24642             2
8            24643             2
9            24644             2
10           24645             2
11           24646             1
12           24648             7
13           24648             8
14           24649             3
15           24651            16
16           24654             1
17           24672             4
18           24679             3
19           24680            10
20           24682             2
21           24695             6
22           24714             2
23           24716            12
24           24718            13
25           24720            15
26           24730            19
27           24751             1
28           24752             1
29        

In [ ]:
import sqlite3

conn = sqlite3.connect('ClassSchedule.db')

# Save dataframes to the database as tables
college_df.to_sql('College', conn, if_exists='replace', index=False)
instructor_df.to_sql('Instructor', conn, if_exists='replace', index=False)
room_df.to_sql('Room', conn, if_exists='replace', index=False)
status_df.to_sql('Status', conn, if_exists='replace', index=False)
class_df.to_sql('Class', conn, if_exists='replace', index=False)
section_df.to_sql('Section', conn, if_exists='replace', index=False)
section_instructor.to_sql('SectionInstructor', conn, if_exists='replace', index=False)

# Close the database connection
conn.close()


In [ ]:
import sqlite3
import pandas as pd

conn = sqlite3.connect('ClassSchedule.db')

# List all tables
tables = pd.read_sql("SELECT name FROM sqlite_master WHERE type='table';", conn)

# Loop through each table and print its contents
for table_name in tables['name']:
    print(f"\nContents of {table_name} table:")
    table_data = pd.read_sql(f"SELECT * FROM {table_name};", conn)
    print(table_data)

conn.close()



Contents of sqlite_sequence table:
Empty DataFrame
Columns: [name, seq]
Index: []

Contents of College table:
  CollegeName  CollegeID
0        ENGR          1

Contents of Instructor table:
   InstructorFirstName InstructorLastName  InstructorID
0                 Erin              Keith             1
1              Bashira        Akter Anima             2
2                Nancy        Latourrette             3
3               Mircea          Nicolescu             4
4                Diana               Moss             5
5               Joshua               Dahl             6
6                 Sara              Davis             7
7                David        Feil-Seifer             8
8                  Lei               Yang             9
9               Shamik           Sengupta            10
10            Christos       Papachristos            11
11             Zachary           Estreito            12
12             William            Doherty            13
13                Hung  